In [ ]:
import os
# os.chdir('/content/CSE144')

Cloning into 'CSE144'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 122 (delta 35), reused 14 (delta 14), pack-reused 71
Receiving objects: 100% (122/122), 1.69 MiB | 4.68 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import random
import torch
from torch import nn, optim
import math
from IPython import display

## column selection
unique_data = pd.read_csv('/content/movie_metadata.csv').drop(['director_facebook_likes', 'actor_3_facebook_likes', 'actor_1_facebook_likes', 'cast_total_facebook_likes', 'actor_2_facebook_likes', 'num_voted_users', 'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link', 'aspect_ratio'], axis=1)
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   color                   5024 non-null   object 
 1   director_name           4939 non-null   object 
 2   num_critic_for_reviews  4993 non-null   float64
 3   duration                5028 non-null   float64
 4   actor_2_name            5030 non-null   object 
 5   gross                   4159 non-null   float64
 6   genres                  5043 non-null   object 
 7   actor_1_name            5036 non-null   object 
 8   movie_title             5043 non-null   object 
 9   actor_3_name            5020 non-null   object 
 10  num_user_for_reviews    5022 non-null   float64
 11  language                5029 non-null   object 
 12  country                 5038 non-null   object 
 13  content_rating          4740 non-null   object 
 14  budget                  4551 non-null   

Data Cleaning

In [ ]:
## getting rid of duplicate titles
unique_data = unique_data.drop_duplicates(subset='movie_title', keep='first')
unique_data = unique_data.drop('movie_title', axis=1)

## missing values
print(unique_data.isnull().sum())

# remove rows that are missing budget or gross income values
unique_data = unique_data.dropna(subset=['gross', 'budget'], how='any')

## Remove rows with'TV'
unique_data = unique_data[~unique_data['content_rating'].str.contains("TV", na=False)]


for column in unique_data.columns:
    if unique_data[column].dtype == 'object':
        unique_data.loc[:, column].fillna("Unknown", inplace=True)
    else:
        # For numerical columns, fill missing values with the median value
        unique_data.loc[:, column].fillna(unique_data[column].median(), inplace=True)

# unique_data.dropna(how="any",axis=0,inplace = True) #this just drops all missing values?
print(unique_data.isnull().sum())

## outliers for gross and budget
Q1 = unique_data[['gross', 'budget']].quantile(0.25)
Q3 = unique_data[['gross', 'budget']].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

unique_data = unique_data[(unique_data['gross'] >= lower_bound['gross']) & (unique_data['gross'] <= upper_bound['gross']) &
                 (unique_data['budget'] >= lower_bound['budget']) & (unique_data['budget'] <= upper_bound['budget'])]

## split genres up
# unique_data['genres'] = unique_data['genres'].apply(lambda a: str(a).replace('|', ' '))
unique_data['genres'] = unique_data['genres'].str.split('|')
# one hot encode genres
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(unique_data['genres'])
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_, index=unique_data.index)

unique_data = unique_data.drop('genres', axis=1).join(genres_df)



unique_data.head()

color                      19
director_name             102
num_critic_for_reviews     49
duration                   15
actor_2_name               13
gross                     863
genres                      0
actor_1_name                7
actor_3_name               23
num_user_for_reviews       21
language                   14
country                     5
content_rating            300
budget                    484
title_year                106
imdb_score                  0
movie_facebook_likes        0
dtype: int64
color                     0
director_name             0
num_critic_for_reviews    0
duration                  0
actor_2_name              0
gross                     0
genres                    0
actor_1_name              0
actor_3_name              0
num_user_for_reviews      0
language                  0
country                   0
content_rating            0
budget                    0
title_year                0
imdb_score                0
movie_facebook_likes      0
d

<ipython-input-3-c0f79faea027>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_data['genres'] = unique_data['genres'].str.split('|')


,color,director_name,num_critic_for_reviews,duration,actor_2_name,gross,actor_1_name,actor_3_name,num_user_for_reviews,language,...,Music,Musical,Mystery,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
50,Color,Baz Luhrmann,490.0,143.0,Elizabeth Debicki,144812796.0,Leonardo DiCaprio,Steve Bisley,753.0,English,...,0,0,0,1,0,0,0,0,0,0
100,Color,Rob Cohen,187.0,106.0,Vin Diesel,144512310.0,Paul Walker,Jordana Brewster,988.0,English,...,0,0,0,0,0,0,0,1,0,0
134,Color,Tim Burton,526.0,113.0,Chloë Grace Moretz,79711678.0,Johnny Depp,Christopher Lee,479.0,English,...,0,0,0,0,0,0,0,0,0,0
157,Color,Dean Parisot,135.0,90.0,Richard Burgi,110332737.0,John Michael Higgins,David Herman,258.0,English,...,0,0,0,0,0,0,0,0,0,0
192,Color,Phillip Noyce,330.0,101.0,Andre Braugher,118311368.0,Angelina Jolie Pitt,August Diehl,514.0,English,...,0,0,1,0,0,0,0,1,0,0


In [ ]:
## One hot encoding
categorical_features = ['color', 'language', 'country', 'content_rating', 'genres']
actor_columns = ['actor_1_name', 'actor_2_name', 'actor_3_name']
director_feature = 'director_name'

actor_counts = unique_data[actor_columns].stack().value_counts()
director_counts = unique_data[director_feature].value_counts()

star_threshold = 0.1  # top 10%

num_stars = int(len(actor_counts) * star_threshold)
stars = set(actor_counts.head(num_stars).index)

def count_stars(row, stars, actor_columns):
    return sum(row[actor] in stars for actor in actor_columns)

unique_data['num_stars'] = unique_data[actor_columns].apply(count_stars, axis=1, stars=stars, actor_columns=actor_columns)
unique_data.drop(columns=actor_columns, inplace=True)

unique_data['director_star_power'] = unique_data['director_name'].map(director_counts)


print("actors and director replaced with star numbers:")
print(unique_data.head)

def calculate_director_avg_gross(director_name, data):
    director_movies = data[data['director_name'] == director_name]
    if not director_movies.empty:
        return director_movies['gross'].mean()
    else:
        return 0


unique_data['director_avg_gross'] = unique_data['director_name'].apply(lambda x: calculate_director_avg_gross(x, unique_data))
unique_data.drop(columns=['director_name'], inplace=True)


#print(unique_data.columns)

# def convert_features_to_one_hot(unique_data, feature_name_list):
#   for feature_name in feature_name_list:
#     df = pd.get_dummies(unique_data, columns=[feature_name])

#   return df

# data_encoded = convert_features_to_one_hot(unique_data, categorical_features)

# data_encoded.columns


def convert_features_to_one_hot(unique_data, feature_name_list):
    print("Initial columns:", unique_data.columns)
    for feature_name in feature_name_list:
        if feature_name in unique_data.columns:
            unique_data = pd.get_dummies(unique_data, columns=[feature_name])
            print(f"Columns after encoding {feature_name}:", unique_data.columns)
        else:
            print(f"Warning: {feature_name} is not in DataFrame columns")
    return unique_data

data_encoded = convert_features_to_one_hot(unique_data, categorical_features)

print("Final columns:", data_encoded.columns)
# print(data_encoded)

actors and director replaced with star numbers:
<bound method NDFrame.head of       color     director_name  num_critic_for_reviews  duration        gross  \
50    Color      Baz Luhrmann                   490.0     143.0  144812796.0   
100   Color         Rob Cohen                   187.0     106.0  144512310.0   
134   Color        Tim Burton                   526.0     113.0   79711678.0   
157   Color      Dean Parisot                   135.0      90.0  110332737.0   
192   Color     Phillip Noyce                   330.0     101.0  118311368.0   
...     ...               ...                     ...       ...          ...   
5033  Color     Shane Carruth                   143.0      77.0     424760.0   
5034  Color  Neill Dela Llana                    35.0      80.0      70071.0   
5035  Color  Robert Rodriguez                    56.0      81.0    2040920.0   
5037  Color      Edward Burns                    14.0      95.0       4584.0   
5042  Color          Jon Gunn             

In [ ]:
## train test splitting
train_set, test_set = train_test_split(data_encoded, test_size=0.1, random_state=42)

print(type(train_set))

print(train_set.head())
print(test_set.head())
print(train_set.num_stars.head(10))
print(train_set.director_star_power.head(10))

<class 'pandas.core.frame.DataFrame'>
      num_critic_for_reviews  duration        gross  num_user_for_reviews  \
2462                    70.0      93.0    3500000.0                 100.0   
2090                    53.0     114.0   50728000.0                 166.0   
2031                   156.0     110.0   25167270.0                 263.0   
938                    140.0     116.0  105807520.0                 357.0   
370                    269.0     121.0   83077470.0                 440.0   

          budget  title_year  imdb_score  movie_facebook_likes  Action  \
2462   2700000.0      1986.0         5.9                     0       1   
2090  22000000.0      1995.0         5.8                     0       1   
2031  24000000.0      2000.0         6.3                   455       0   
938   50000000.0      2003.0         6.4                     0       0   
370   75000000.0      2008.0         7.1                     0       0   

      Adventure  ...  content_rating_M  content_rating

Regression Models

In [ ]:
import torch
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np

#accuracy function
def within_percentage_accuracy(outputs, targets, percentage=0.10):
    with torch.no_grad():
        #convert torch tensors
        outputs = torch.tensor(outputs)
        targets = torch.tensor(targets.values)

        deviation = percentage * targets
        lower_bounds = targets - deviation
        upper_bounds = targets + deviation

        correct = torch.logical_and(outputs >= lower_bounds, outputs <= upper_bounds)
        accuracy = torch.mean(correct.float())
    return accuracy.item() * 100


print(data_encoded.columns)
independent_vars = train_set[data_encoded.columns.drop('gross')]
dependent_var = train_set['gross']

X_train, X_test, y_train, y_test = train_test_split(independent_vars, dependent_var, test_size=0.2, random_state=42)

model = LinearRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)

ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)

lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Linear Regression - MSE: {mse}, MAE: {mae}, R-squared: {r2}, RMSE: {rmse}")

mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)

print(f"Ridge Regression - MSE: {mse_ridge}, MAE: {mae_ridge}, R-squared: {r2_ridge}, RMSE: {rmse_ridge}")

mse_lasso = mean_squared_error(y_test, y_pred_lasso)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mse_lasso)

print(f"Lasso Regression - MSE: {mse_lasso}, MAE: {mae_lasso}, R-squared: {r2_lasso}, RMSE: {rmse_lasso}")

accuracy_linear = within_percentage_accuracy(y_pred, y_test, percentage=0.4)
accuracy_ridge = within_percentage_accuracy(y_pred_ridge, y_test, percentage=0.4)
accuracy_lasso = within_percentage_accuracy(y_pred_lasso, y_test, percentage=0.4)

print(f"Linear Regression - Accuracy within 10%: {accuracy_linear}%")
print(f"Ridge Regression - Accuracy within 10%: {accuracy_ridge}%")
print(f"Lasso Regression - Accuracy within 10%: {accuracy_lasso}%")


Index(['num_critic_for_reviews', 'duration', 'gross', 'num_user_for_reviews',
       'budget', 'title_year', 'imdb_score', 'movie_facebook_likes', 'Action',
       'Adventure',
       ...
       'content_rating_M', 'content_rating_NC-17', 'content_rating_Not Rated',
       'content_rating_PG', 'content_rating_PG-13', 'content_rating_Passed',
       'content_rating_R', 'content_rating_Unknown', 'content_rating_Unrated',
       'content_rating_X'],
      dtype='object', length=133)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.63748e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Linear Regression - MSE: 438742494937056.94, MAE: 14802097.054077286, R-squared: 0.6433362805881839, RMSE: 20946180.91531382
Ridge Regression - MSE: 435776108141328.56, MAE: 14767997.201848933, R-squared: 0.6457477236555584, RMSE: 20875251.091695365
Lasso Regression - MSE: 440019992994330.6, MAE: 14830284.366125463, R-squared: 0.6422977734595001, RMSE: 20976653.52229308
Linear Regression - Accuracy within 10%: 46.22950851917267%
Ridge Regression - Accuracy within 10%: 46.06557488441467%
Lasso Regression - Accuracy within 10%: 46.393442153930664%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.742e+17, tolerance: 2.909e+14
  model = cd_fast.enet_coordinate_descent(


In [ ]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

train_set, val_set = train_test_split(
    train_set, test_size=0.222, random_state=42)  #0.25 x 0.8 = 0.2

train_features = train_set.drop(columns=['gross']).values.astype(int)
train_targets = train_set['gross'].values

val_features = val_set.drop(columns=['gross']).values.astype(int)
val_targets = val_set['gross'].values

test_features = test_set.drop(columns=['gross']).values.astype(int)
test_targets = test_set['gross'].values

print(type(train_features))
print(train_features)





<class 'numpy.ndarray'>
[[ 200  152 1103 ...    0    0    0]
 [  16   95  158 ...    0    0    0]
 [ 113  102  301 ...    0    0    0]
 ...
 [  26  124   74 ...    0    0    0]
 [  81  121   94 ...    0    0    0]
 [ 169  110  374 ...    0    0    0]]


Feed Forward Neural Network

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
train_features = torch.tensor(train_features, dtype=torch.float32)
train_targets = torch.tensor(train_targets, dtype=torch.float32)

val_features = torch.tensor(val_features, dtype=torch.float32)
val_targets = torch.tensor(val_targets, dtype=torch.float32)

test_features = torch.tensor(test_features, dtype=torch.float32)
test_targets = torch.tensor(test_targets, dtype=torch.float32)

class MovieDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = torch.tensor(targets, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

train_dataset = MovieDataset(train_features, train_targets)
val_dataset = MovieDataset(val_features, val_targets)
test_dataset = MovieDataset(test_features, test_targets)

batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc0 = nn.Linear(train_features.shape[1], 512)
        self.bn0 = nn.BatchNorm1d(512)
        self.fc1 = nn.Linear(512, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.fc5 = nn.Linear(32, 1)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.2)

    def forward(self, x):
        # residual = self.residual_fc(x)
        x = F.relu(self.bn0(self.fc0(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn4(self.fc4(x)))
        x = self.fc5(x)
        return x

model = Net()

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1.0, weight_decay=1e-5)  #L2 regularization
# lr = 0.001
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=10, min_lr=0.0001)


train_losses = []
val_losses = []

def within_percentage_accuracy_FF(outputs, targets, percentage=0.20):
    with torch.no_grad():
        deviation = percentage * targets
        lower_bounds = targets - deviation
        upper_bounds = targets + deviation

        correct = torch.logical_and(outputs >= lower_bounds, outputs <= upper_bounds)
        accuracy = torch.mean(correct.float())
    return accuracy.item() * 100

train_accuracies = []
val_accuracies = []

num_epochs = 200
best_val_loss = float('inf')
patience, trials = 10, 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_accuracy = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        # print("Predictions:", outputs[:5])
        # print("Actual:", targets[:5])
        loss = criterion(outputs, targets)
        # print("loss", loss)
        # print("loss item", loss.item())
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=1.0)
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

        total_accuracy += within_percentage_accuracy_FF(outputs, targets)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    train_losses.append(epoch_loss)
    # print(train_losses)
    epoch_accuracy = total_accuracy / len(train_loader)
    train_accuracies.append(epoch_accuracy)

    # Validation phase
    model.eval()
    val_loss = 0.0
    total_val_accuracy = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            # print("val loss", loss)
            # print("val loss item", loss.item())
            val_loss += loss.item() * inputs.size(0)
            total_val_accuracy += within_percentage_accuracy_FF(outputs, targets)

        val_loss /= len(val_loader.dataset)
        print(f'Validation MSE: {val_loss:.4f}')
        val_losses.append(val_loss)
        val_accuracy = total_val_accuracy / len(val_loader)
        val_accuracies.append(val_accuracy)

        scheduler.step(val_loss)

    # # Early stopping check
    # if val_loss < best_val_loss:
    #     best_val_loss = val_loss
    #     trials = 0
    # else:
    #     trials += 1
    #     if trials >= patience:
    #         print("Early stopping triggered.")
    #         break
model.eval()
test_loss = 0.0
total_test_accuracy = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        # print("test loss", loss)
        # print("test loss item", loss.item())
        test_loss += loss.item() * inputs.size(0)
        total_test_accuracy += within_percentage_accuracy_FF(outputs, targets)
    test_loss /= len(test_loader.dataset)
    print(f'Final Test MSE: {test_loss:.4f}')
    test_accuracy = total_test_accuracy / len(test_loader)
    print(f'Final Test Accuracy: {test_accuracy:.4f}')








<ipython-input-11-1bc1725ac09e>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_features = torch.tensor(train_features, dtype=torch.float32)
<ipython-input-11-1bc1725ac09e>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_targets = torch.tensor(train_targets, dtype=torch.float32)
<ipython-input-11-1bc1725ac09e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_features = torch.tensor(val_features, dtype=torch.float32)
<ipython-input-11-1bc1725ac09e>:8: UserWarning: To copy construct from a tensor, it is recommended to use 

Epoch 1/200, Loss: 2311873078992274.5000
Validation MSE: 2356675346918341.0000
Epoch 2/200, Loss: 2309488802000571.5000
Validation MSE: 2351384883363453.0000
Epoch 3/200, Loss: 2304334028922562.5000
Validation MSE: 2345660812091065.5000
Epoch 4/200, Loss: 2296266404090263.5000
Validation MSE: 2334299732641196.0000
Epoch 5/200, Loss: 2285596751770021.5000
Validation MSE: 2320817477804044.0000
Epoch 6/200, Loss: 2272275021424153.0000
Validation MSE: 2305028352099199.5000
Epoch 7/200, Loss: 2256498125171527.0000
Validation MSE: 2280747013014036.5000
Epoch 8/200, Loss: 2238060871040112.2500
Validation MSE: 2262353322313809.5000
Epoch 9/200, Loss: 2216180541528164.2500
Validation MSE: 2233019678623777.2500
Epoch 10/200, Loss: 2192837909190645.7500
Validation MSE: 2216518498332097.2500
Epoch 11/200, Loss: 2166528713336925.2500
Validation MSE: 2177910115032325.7500
Epoch 12/200, Loss: 2137736621243228.0000
Validation MSE: 2153267317494722.0000
Epoch 13/200, Loss: 2107210085734389.7500
Validat

In [ ]:

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
print(train_losses)

plt.title('Train Loss Progression')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.plot(val_losses, label='Validation Loss')
print(val_losses)

plt.scatter(len(train_losses), test_loss, color='red', label='Test Loss', zorder=5)

plt.title('Val Loss Progression')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


# losses without the super high val loss at the start, which I dont know the cause of
plt.figure(figsize=(10, 5))
plt.ylim(top=train_losses[0])
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.scatter(len(train_losses), test_loss, color='red', label='Test Loss', zorder=5)

plt.title('Combined Loss Progression')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.show()


plt.figure(figsize=(10, 5))
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')

plt.scatter(len(train_accuracies), test_accuracy, color='red', label='Test Accuracy', zorder=5)

plt.title('Accuracy Progression')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'plt' is not defined

Model using Keras

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, Input, Concatenate, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler

unique_data = pd.read_csv('/content/movie_metadata.csv')

unique_data = unique_data.drop(['director_facebook_likes', 'actor_3_facebook_likes', 'actor_1_facebook_likes',
                                'cast_total_facebook_likes', 'actor_2_facebook_likes', 'num_voted_users',
                                'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link', 'aspect_ratio'], axis=1)

#duplicate titles
unique_data = unique_data.drop_duplicates(subset='movie_title', keep='first')
unique_data = unique_data.drop('movie_title', axis=1)

unique_data = unique_data.dropna(subset=['gross', 'budget'], how='any')
unique_data = unique_data[~unique_data['content_rating'].str.contains("TV", na=False)]

for column in unique_data.columns:
    if unique_data[column].dtype == 'object':
        unique_data[column].fillna("Unknown", inplace=True)
    else:
        unique_data[column].fillna(unique_data[column].median(), inplace=True)

# Outliers for gross and budget
Q1 = unique_data[['gross', 'budget']].quantile(0.25)
Q3 = unique_data[['gross', 'budget']].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

unique_data = unique_data[(unique_data['gross'] >= lower_bound['gross']) & (unique_data['gross'] <= upper_bound['gross']) &
                          (unique_data['budget'] >= lower_bound['budget']) & (unique_data['budget'] <= upper_bound['budget'])]

unique_data['genres'] = unique_data['genres'].str.split('|')
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(unique_data['genres'])
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_, index=unique_data.index)

unique_data = unique_data.drop('genres', axis=1).join(genres_df)

categorical_features = ['color', 'language', 'country', 'content_rating']
target_column = 'gross'

numeric_features = unique_data.select_dtypes(include=[float, int]).columns.tolist()
numeric_features = [feature for feature in numeric_features if feature not in categorical_features + [target_column]]

for feature in categorical_features:
    unique_data[feature] = unique_data[feature].astype('category').cat.codes

scaler = StandardScaler()
unique_data[numeric_features] = scaler.fit_transform(unique_data[numeric_features])

X = unique_data.drop(columns=[target_column])
y = unique_data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

input_layers = []
embedding_layers = []

for feature in categorical_features:
    vocab_size = unique_data[feature].nunique()
    embedding_dim = 8
    input_layer = Input(shape=(1,), name=f"input_{feature}")
    input_layers.append(input_layer)
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1, name=f"embedding_{feature}")(input_layer)
    embedding_layer = Flatten()(embedding_layer)
    embedding_layers.append(embedding_layer)

numeric_input = Input(shape=(len(numeric_features),), name='numeric_input')
input_layers.append(numeric_input)
embedding_layers.append(numeric_input)

concatenated = Concatenate()(embedding_layers)

x = Dense(256, activation='relu')(concatenated)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(1)(x)

model = tf.keras.Model(inputs=input_layers, outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()

train_inputs = {f"input_{feature}": X_train[feature] for feature in categorical_features}
train_inputs['numeric_input'] = X_train[numeric_features]

test_inputs = {f"input_{feature}": X_test[feature] for feature in categorical_features}
test_inputs['numeric_input'] = X_test[numeric_features]

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(train_inputs, y_train, epochs=200, batch_size=32, validation_data=(test_inputs, y_test), callbacks=[early_stopping])

loss, mae = model.evaluate(test_inputs, y_test)
print(f'Test Loss: {loss}')
print(f'Test MAE: {mae}')

def custom_accuracy(y_true, y_pred, threshold=0.1):
    diff = tf.abs(y_true - y_pred)
    return tf.reduce_mean(tf.cast(diff <= threshold * y_true, tf.float32))

y_pred = model.predict(test_inputs).flatten()
custom_acc = custom_accuracy(y_test.values, y_pred)
print(f'Custom Accuracy: {custom_acc.numpy()}')

r2 = r2_score(y_test, y_pred)
print(f'R^2: {r2}')


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_color (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 input_language (InputLayer  [(None, 1)]                  0         []                            
 )                                                                                                
                                                                                                  
 input_country (InputLayer)  [(None, 1)]                  0         []                            
                                                                                                  
 input_content_rating (Inpu  [(None, 1)]                  0         []                        